## Day 17: Reservoir Research

https://adventofcode.com/2018/day/17

### Part 1

This looks potentially frustrating.

I can't think of a clever approach so let's try following the instructions. Represent the reservoir as a `defaultdict`, with coordinates as keys, `'#'` as clay, `'|'` as flowing water, `'~'` as settled water and `'.'` as sand, the default.

Maintain a list of flowing water to process. While the list exists take a square of flowing water.

If the square below the flowing water is empty, if it is within the reservoir bounds mark the square as flowing water and add it to the stack. 

If the square below the flowing water is flowing water, do nothing.

If the square below the flowing water is settled water or clay, then it gets complicated. Go left and right while the squares are empty and the squares below are settled water or clay. If both left and right hit clay before being unsupported, set all the squares between left and right to settled water and add the square above the original square to the processing list. Otherwise set all the squares between left and right to flowing; set whichever ends don't hit clay to the processing list.

When the processing list is empty count the number of squares assigned to water.


I'll follow standard Advent of Code software engineering procedure and write one massive function and debug with `print` statements.

First parse the input.

In [1]:
from parse import parse
from collections import defaultdict


def parse_reservoir(input):
    reservoir = defaultdict(lambda: '.')
    
    for line in input:
        variable, a, _, b1, b2 = parse('{}={:d}, {}={:d}..{:d}', line).fixed
        
        if variable == 'x':
            for y in range(b1, b2 + 1):
                reservoir[(a, y)] = '#'
        else: 
            for x in range(b1, b2 + 1):
                reservoir[(x, a)] = '#'
                
    return reservoir


def print_reservoir(reservoir):
    xs, ys = zip(*reservoir)
    
    for y in range(min(ys), max(ys) + 1):
        line = ''
        
        for x in range(min(xs), max(xs) + 1):
            line = line + reservoir[(x, y)]
            
        print(line)


In [2]:
test_input = '''x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504'''.splitlines()
        
test_reservoir = parse_reservoir(test_input)

print_reservoir(test_reservoir)

...........#
#..#.......#
#..#..#.....
#..#..#.....
#.....#.....
#.....#.....
#######.....
............
............
...#.....#..
...#.....#..
...#.....#..
...#######..


In [34]:
from itertools import count
from copy import deepcopy


def flow(reservoir):
    reservoir = deepcopy(reservoir)
    
    min_y = min(y for _, y in reservoir)
    max_y = max(y for _, y in reservoir)
    
    reservoir[(500, min_y)] = '|'
    processing = [(500, min_y)]
    
    while processing:
        x, y = processing.pop()
        
        if y + 1 <= max_y and y >= min_y:  
            # If there's nothing below, go down
            if reservoir[(x, y + 1)] == '.':
                reservoir[(x, y + 1)] = '|'
                processing.append((x, y + 1))
                
            # Otherwise flow left and right
            elif reservoir[(x, y + 1)] in '~#':
                # Go left until unsupported or hit clay
                left = next(a for a in count(x - 1, -1)
                            if reservoir[(a, y)] == '#' 
                            or reservoir[(a, y + 1)] not in '~#')
                
                # Ditto for right
                right = next(a for a in count(x + 1)
                            if reservoir[(a, y)] == '#' 
                             or reservoir[(a, y + 1)] not in '~#')

                # If clay either side then the water is settled
                if reservoir[(left, y)] == '#' and reservoir[(right, y)] == '#':
                    for a in range(left + 1, right):
                        reservoir[(a, y)] = '~'                        
                    processing.append((x, y - 1))
                    
                # Otherwise water flows to the sides and drops down
                # if there isn't a clay wall
                else:
                    for a in range(left + 1, right):
                        reservoir[(a, y)] = '|'

                    if reservoir[(left, y)] == '.':
                        reservoir[(left, y)] = '|'
                        processing.append((left, y))

                    if reservoir[(right, y)] == '.':
                        reservoir[(right, y)] = '|'
                        processing.append((right, y))
                    
    return reservoir

In [35]:
print_reservoir(flow(test_reservoir))

.....|.....#
#..#||||...#
#..#~~#|....
#..#~~#|....
#~~~~~#|....
#~~~~~#|....
#######|....
.......|....
..|||||||||.
..|#~~~~~#|.
..|#~~~~~#|.
..|#~~~~~#|.
..|#######|.


That actually looks right.

In [36]:
def count_water(reservoir):
    return sum(1 if x in '~|' else 0 for x in reservoir.values())

assert count_water(flow(test_reservoir)) == 57

In [37]:
reservoir = parse_reservoir(open('input', 'r'))

%time count_water(flow(reservoir))

CPU times: user 280 ms, sys: 0 ns, total: 280 ms
Wall time: 280 ms


27331

That was surprisingly painless.

### Part 2

Only a minor change needed here.

In [41]:
def count_settled_water(reservoir):
    return sum(1 if x == '~' else 0 for x in reservoir.values())

assert count_settled_water(flow(test_reservoir)) == 29

In [42]:
count_settled_water(flow(reservoir))

22245